Sprint 11 - Proyecto

Descripción del proyecto
Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
Crear un modelo para predecir el volumen de reservas en pozos nuevos.
Elegir los pozos petrolíferos que tienen los valores estimados más altos.
Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Instrucciones del proyecto
Descarga y prepara los datos. Explica el procedimiento.

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import numpy as np


# Cargar los datos en DataFrames
df_0 = pd.read_csv('geo_data_0.csv')
df_1 = pd.read_csv('geo_data_1.csv')
df_2 = pd.read_csv('geo_data_2.csv')

# Mostrar las primeras filas de cada conjunto de datos
print(df_0.head(), "\n")
print(df_1.head(), "\n")
print(df_2.head(), "\n")

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647 

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305 

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746 



2. Explorar la estructura de los datos
Verificamos las columnas y tipos de datos con .info()

In [17]:
print(df_0.info(), "\n")
print(df_1.info(), "\n")
print(df_2.info(), "\n")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Colu

Explicación:

Revisamos si todas las columnas tienen datos del tipo esperado (float64 para características numéricas y object para IDs).
Nos aseguramos de que todas las regiones tienen el mismo formato.

3. Verificar valores faltantes y duplicados
Buscamos valores nulos y duplicados que puedan afectar el modelo.


In [18]:
print(df_0.isnull().sum(), "\n")
print(df_1.isnull().sum(), "\n")
print(df_2.isnull().sum(), "\n")

print(df_0.duplicated().sum(), "\n")
print(df_1.duplicated().sum(), "\n")
print(df_2.duplicated().sum(), "\n")


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64 

0 

0 

0 



Explicación:

Si hay valores nulos, necesitaremos decidir si eliminarlos o rellenarlos con algún valor.
Si hay duplicados, podrían indicar errores en la recopilación de datos.
4. Describir los datos estadísticamente
Para entender la distribución de los valores, usamos .describe()

In [19]:
print(df_0.describe(), "\n")
print(df_1.describe(), "\n")
print(df_2.describe(), "\n")


                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347 

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%     

Explicación:

Nos ayuda a ver los valores mínimos, máximos, media y desviación estándar.
Analizaremos si los datos tienen valores extremos que puedan afectar el modelo.
Conclusión
Después de este análisis, podremos determinar:
✅ Si los datos están completos y listos para usar.
✅ Si hay problemas como valores nulos o duplicados.
✅ Cómo se distribuyen las variables en cada región.


Entrena y prueba el modelo para cada región en geo_data_0.csv:

Divide los datos en un conjunto de entrenamiento y un conjunto de validación en una proporción de 75:25
2.Entrena un modelo de regresión lineal.
3.Entrena un modelo de regresión lineal.
4.Hace predicciones en el conjunto de validación.
5.Calcula y muestra el RMSE y el volumen medio de reservas predicho.

In [20]:


# Función para entrenar el modelo y calcular métricas
def train_and_evaluate(df, region_name):
    print(f"\nRegión {region_name}")
    
    # Separar características y variable objetivo
    features = df[['f0', 'f1', 'f2']]
    target = df['product']

    # Dividir los datos en entrenamiento (75%) y validación (25%)
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42
    )

    # Entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(features_train, target_train)

    # Hacer predicciones en el conjunto de validación
    predictions = model.predict(features_valid)

    # Calcular RMSE
    mse = mean_squared_error(target_valid, predictions)
    rmse = np.sqrt(mse)

    # Mostrar resultados
    print(f"RMSE del modelo: {rmse:.2f}")
    print(f"Volumen medio de reservas predicho: {predictions.mean():.2f}")
    
    return model, predictions, target_valid  # Retornar modelo y predicciones para su uso posterior

# Aplicar a cada región
model_0, predictions_0, target_valid_0 = train_and_evaluate(df_0, "0")
model_1, predictions_1, target_valid_1 = train_and_evaluate(df_1, "1")
model_2, predictions_2, target_valid_2 = train_and_evaluate(df_2, "2")



Región 0
RMSE del modelo: 37.76
Volumen medio de reservas predicho: 92.40

Región 1
RMSE del modelo: 0.89
Volumen medio de reservas predicho: 68.71

Región 2
RMSE del modelo: 40.15
Volumen medio de reservas predicho: 94.77


Expliacion 

 Calcular el error cuadrático medio (MSE)

mean_squared_error(y_real, y_pred) calcula el error cuadrático medio (MSE), que mide qué tan lejos están las predicciones de los valores reales.

2️⃣ Obtener la raíz del error cuadrático medio (RMSE)

La raíz cuadrada del MSE nos da el error cuadrático medio raíz (RMSE).


3️⃣ Mostrar resultados

rmse:.2f formatea el número con 2 decimales.
predictions.mean() nos da el promedio de reservas predichas, lo que nos ayuda a entender si el modelo sobreestima o subestima el valor real.



**3 Preparate para el calculo de ganancias**


**Paso 3.1: Almacenar valores clave**

Definimos las variables necesarias:

In [21]:
# Costo total de inversión en millones de dólares
total_budget = 100_000_000  

# Número de pozos seleccionados
num_selected_wells = 200  

# Precio por barril en dólares
price_per_barrel = 4.5  

# Ingreso por cada unidad (miles de barriles)
revenue_per_unit = price_per_barrel * 1000  

# Punto de equilibrio por pozo
break_even_units = total_budget / (num_selected_wells * revenue_per_unit) 

print(f"Cada pozo debe producir al menos {break_even_units:.1f} unidades (miles de barriles) para evitar pérdidas.")


Cada pozo debe producir al menos 111.1 unidades (miles de barriles) para evitar pérdidas.


**Paso 3.2: Comparación con la cantidad media de reservas**

Cada pozo debe generar 111.1 unidades para evitar pérdidas.
El volumen medio de reservas en la región 0 nos dirá si en promedio cada pozo está por encima o por debajo de este umbral.


Si el valor medio de reservas en la región es menor a 111.1 unidades, perforar al azar sería una estrategia arriesgada. En este caso, la clave será seleccionar solo los mejores pozos.

In [22]:
# Calcular la media de reservas en cada región
mean_reserves_0 = df_0['product'].mean()
mean_reserves_1 = df_1['product'].mean()
mean_reserves_2 = df_2['product'].mean()

print(f"Media de reservas - Región 0: {mean_reserves_0:.1f} unidades")
print(f"Media de reservas - Región 1: {mean_reserves_1:.1f} unidades")
print(f"Media de reservas - Región 2: {mean_reserves_2:.1f} unidades")


Media de reservas - Región 0: 92.5 unidades
Media de reservas - Región 1: 68.8 unidades
Media de reservas - Región 2: 95.0 unidades


**3.3 Conclusiones sobre la preparación**

A partir de estos valores, podemos analizar si las regiones tienen suficiente producción promedio para cubrir el umbral de rentabilidad. Si la media de reservas de una región es menor que el punto de equilibrio, entonces dependeremos de seleccionar solo los mejores pozos.

**Paso 4.1: Seleccionar los 200 pozos con las predicciones más altas**

Ordenamos las predicciones y seleccionamos los 200 pozos con el mayor valor predicho.

In [23]:


num_selected_wells = 200  # Número de pozos a seleccionar

# Dividir los datos en entrenamiento (75%) y validación (25%) para cada región
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    df_0[['f0', 'f1', 'f2']], df_0['product'], test_size=0.25, random_state=42
)
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    df_1[['f0', 'f1', 'f2']], df_1['product'], test_size=0.25, random_state=42
)
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    df_2[['f0', 'f1', 'f2']], df_2['product'], test_size=0.25, random_state=42
)

# Entrenar modelos de regresión lineal para cada región
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)

model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)

# Función para seleccionar los 200 mejores pozos por predicción
def select_top_wells(features, target, model):
    predictions = model.predict(features)
    top_indices = np.argsort(predictions)[-num_selected_wells:]  # Seleccionar los 200 mejores
    return target.iloc[top_indices], predictions[top_indices]

# Seleccionar los mejores pozos en cada región
top_target_0, top_predictions_0 = select_top_wells(features_valid_0, target_valid_0, model_0)
top_target_1, top_predictions_1 = select_top_wells(features_valid_1, target_valid_1, model_1)
top_target_2, top_predictions_2 = select_top_wells(features_valid_2, target_valid_2, model_2)

# Mostrar información de los 200 mejores pozos por región
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 0: {top_target_0.mean():.2f}")
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 1: {top_target_1.mean():.2f}")
print(f"Volumen medio de reservas en los mejores 200 pozos - Región 2: {top_target_2.mean():.2f}")


Volumen medio de reservas en los mejores 200 pozos - Región 0: 148.43
Volumen medio de reservas en los mejores 200 pozos - Región 1: 137.95
Volumen medio de reservas en los mejores 200 pozos - Región 2: 139.98


**Paso 4.2: Calcular el volumen objetivo de reservas**

Sumamos las reservas reales (actual) de los 200 mejores pozos para ver cuántas reservas reales tienen.

In [24]:
# Calcular el volumen total de reservas en los 200 mejores pozos de cada región
total_reserves_0 = top_target_0.sum()
total_reserves_1 = top_target_1.sum()
total_reserves_2 = top_target_2.sum()

# Mostrar los resultados
print(f"Volumen total de reservas en los 200 mejores pozos - Región 0: {total_reserves_0:.2f}")
print(f"Volumen total de reservas en los 200 mejores pozos - Región 1: {total_reserves_1:.2f}")
print(f"Volumen total de reservas en los 200 mejores pozos - Región 2: {total_reserves_2:.2f}")


Volumen total de reservas en los 200 mejores pozos - Región 0: 29686.98
Volumen total de reservas en los 200 mejores pozos - Región 1: 27589.08
Volumen total de reservas en los 200 mejores pozos - Región 2: 27996.83


**Paso 4.3: Calcular la ganancia potencial**


Utilizamos la fórmula:

Ganancia=Total de reservas×4500−100,000,000


In [25]:
# Definir constantes
REVENUE_PER_UNIT = 4500  # Ingreso por unidad de reserva
INVESTMENT = 100_000_000  # Inversión total en dólares

# Calcular la ganancia potencial en cada región
profit_0 = total_reserves_0 * REVENUE_PER_UNIT - INVESTMENT
profit_1 = total_reserves_1 * REVENUE_PER_UNIT - INVESTMENT
profit_2 = total_reserves_2 * REVENUE_PER_UNIT - INVESTMENT

# Mostrar los resultados
print(f"Ganancia potencial - Región 0: ${profit_0:,.2f}")
print(f"Ganancia potencial - Región 1: ${profit_1:,.2f}")
print(f"Ganancia potencial - Región 2: ${profit_2:,.2f}")


Ganancia potencial - Región 0: $33,591,411.14
Ganancia potencial - Región 1: $24,150,866.97
Ganancia potencial - Región 2: $25,985,717.59


**Conclusión**

Volumen objetivo de reservas:

Al sumar las reservas reales de los 200 mejores pozos en cada región, observamos diferencias significativas en la capacidad productiva de cada zona.
Ganancia potencial:

Aplicando la fórmula de ganancias, encontramos que algunas regiones pueden generar un margen positivo, mientras que otras podrían no ser rentables.
Si una región tiene un beneficio negativo, significa que la inversión inicial de $100M no se recupera.
La región con la mayor ganancia potencial es la que tiene el mayor total de reservas entre sus 200 mejores pozos.
Recomendación:

La región con la mayor ganancia positiva es la opción más atractiva para la inversión en pozos petroleros.
Si dos o más regiones tienen beneficios similares, es importante evaluar otros factores como riesgo de pérdidas, estabilidad de las predicciones y variabilidad en las reservas antes de tomar una decisión final.

**Paso 5.1: Aplicar Bootstrapping**

Generamos 1000 muestras aleatorias con reemplazo y calculamos la ganancia para cada una.

In [26]:

# Definir la cantidad de simulaciones
num_bootstrap_samples = 1000

# Función para calcular la ganancia con bootstrapping
def bootstrap_profit(target, num_samples=500, num_selected_wells=200):
    profits = []
    
    for _ in range(num_bootstrap_samples):
        sample = target.sample(n=num_samples, replace=True)  # Seleccionar una muestra aleatoria con reemplazo
        top_sample = sample.nlargest(num_selected_wells)  # Seleccionar los 200 mejores pozos
        total_reserves = top_sample.sum()  # Sumar las reservas reales
        profit = total_reserves * 4500 - 100_000_000  # Aplicar la fórmula de ganancia
        profits.append(profit)
    
    return np.array(profits)

# Aplicar bootstrapping a cada región
bootstrap_profits_0 = bootstrap_profit(top_target_0)
bootstrap_profits_1 = bootstrap_profit(top_target_1)
bootstrap_profits_2 = bootstrap_profit(top_target_2)

# Mostrar un resumen de las ganancias obtenidas
print(f"Media de ganancia - Región 0: ${bootstrap_profits_0.mean():,.2f}")
print(f"Media de ganancia - Región 1: ${bootstrap_profits_1.mean():,.2f}")
print(f"Media de ganancia - Región 2: ${bootstrap_profits_2.mean():,.2f}")


Media de ganancia - Región 0: $52,497,290.00
Media de ganancia - Región 1: $24,150,866.97
Media de ganancia - Región 2: $49,312,422.99


**Paso 5.2: Calcular estadísticas clave**


Extraemos el beneficio promedio, el intervalo de confianza del 95%, y el riesgo de pérdida (casos donde la ganancia es negativa).

In [27]:
# Función para calcular estadísticas clave
def calculate_statistics(profits):
    mean_profit = profits.mean()  # Beneficio promedio
    lower_bound = np.percentile(profits, 2.5)  # Límite inferior del intervalo de confianza del 95%
    upper_bound = np.percentile(profits, 97.5)  # Límite superior del intervalo de confianza del 95%
    risk_of_loss = (profits < 0).mean()  # Proporción de muestras con pérdida (ganancia negativa)

    return mean_profit, lower_bound, upper_bound, risk_of_loss

# Calcular estadísticas para cada región
stats_0 = calculate_statistics(bootstrap_profits_0)
stats_1 = calculate_statistics(bootstrap_profits_1)
stats_2 = calculate_statistics(bootstrap_profits_2)

# Mostrar resultados
print(f"Región 0: Media de ganancia: ${stats_0[0]:,.2f}, IC 95%: (${stats_0[1]:,.2f}, ${stats_0[2]:,.2f}), Riesgo de pérdida: {stats_0[3]:.2%}")
print(f"Región 1: Media de ganancia: ${stats_1[0]:,.2f}, IC 95%: (${stats_1[1]:,.2f}, ${stats_1[2]:,.2f}), Riesgo de pérdida: {stats_1[3]:.2%}")
print(f"Región 2: Media de ganancia: ${stats_2[0]:,.2f}, IC 95%: (${stats_2[1]:,.2f}, ${stats_2[2]:,.2f}), Riesgo de pérdida: {stats_2[3]:.2%}")


Región 0: Media de ganancia: $52,497,290.00, IC 95%: ($50,894,980.98, $53,967,363.90), Riesgo de pérdida: 0.00%
Región 1: Media de ganancia: $24,150,866.97, IC 95%: ($24,150,866.97, $24,150,866.97), Riesgo de pérdida: 0.00%
Región 2: Media de ganancia: $49,312,422.99, IC 95%: ($47,435,999.69, $51,254,532.78), Riesgo de pérdida: 0.00%


**Paso 5.3: Conclusión**

Conclusión del Análisis con Bootstrapping
Tras aplicar bootstrapping con 1000 simulaciones, obtuvimos el beneficio promedio, el intervalo de confianza del 95% y el riesgo de pérdida para cada región.

Resultados clave:
La región con la mayor ganancia promedio es la mejor candidata para la inversión.
Si alguna región tiene un riesgo de pérdida superior al 2.5%-5%, es una opción más riesgosa.
El intervalo de confianza del 95% nos indica el rango en el que es probable que se encuentren las ganancias reales.
Decisión Final:
Si una región tiene un beneficio alto y un bajo riesgo de pérdida, es la mejor opción para invertir. En cambio, si los beneficios son similares entre regiones, pero una tiene menos riesgo, esa sería la más segura.

Con estos resultados, podemos tomar una decisión basada en datos para maximizar la rentabilidad y minimizar riesgos. 

**Propuesta de Región para el Desarrollo de Pozos Petrolíferos**

Con base en los análisis realizados, la región óptima para el desarrollo de pozos petrolíferos es la que mostró:

✅ Mayor ganancia promedio en el bootstrapping.
✅ Menor riesgo de pérdida (probabilidad de obtener ganancias negativas).
✅ Intervalo de confianza del 95% más alto, lo que sugiere estabilidad en las ganancias.

¿Coincide con la elección del paso 4.3?
Si la región seleccionada en el paso 4.3 (basado en la predicción de los 200 mejores pozos) es la misma que resultó más rentable en el análisis de riesgos del bootstrapping, entonces sí coincide. Esto validaría que nuestras predicciones eran acertadas.

Sin embargo, si la región con más reservas en el paso 4.3 tiene un alto riesgo de pérdida en el bootstrapping, podría ser mejor reconsiderar la inversión y optar por una región más estable.

Justificación Final
La región seleccionada maximiza el beneficio esperado mientras minimiza el riesgo. Al tomar en cuenta tanto las predicciones iniciales como el análisis de simulaciones, podemos hacer una inversión más informada y estratégica